In [1]:
import numpy as np
import pandas as pd
import glob
import sys,os

In [2]:
from pathlib import Path

### API

In [4]:
sys.path.append('../../')

In [5]:
from utils.bbox.conversation import bbox_coco2voc, bbox_voc2yolo

In [6]:
def bbox_voc2yolo_df(df):
    w =  df['width'].values
    h =  df['height'].values
    w_bbox = df['bx3'].values - df['bx1'].values
    h_bbox = df['bx4'].values - df['bx2'].values
    x_c = df['bx1'].values + w_bbox/2
    y_c = df['bx2'].values + h_bbox/2
    df.loc[:, 'bx1'] = x_c/w
    df.loc[:, 'bx2'] = y_c/h
    df.loc[:, 'bx3'] = w_bbox/w
    df.loc[:, 'bx4'] = h_bbox/h    
    return df

### Dataset Information

In [3]:
root_nas = Path('/nas/chenyi/workspace/myPyModule/dataset/data')

In [8]:
!ls $root_data/

bboxes	dict  @eaDir  slink  watoo


In [9]:
!ls $root_data/bboxes

dadet100k     @eaDir  fashionpedia  hwpedia  tuneDataset
deepfashion2  exps    haowei34k     query


In [10]:
!ls $root_data/bboxes/query

@eaDir				  shoeBag10k0530    wholebody5k0610
haowei34k_imgs_info.csv		  shoeBag5k0515
haowei34k_voc_bboxes_relabel.csv  sys_error_sample


In [11]:
!ls $root_data/bboxes/query/shoeBag10k0530

shoe_bag11k_map.xlsx  shoe_bag11k_voc_Jul.csv  shoe_bag11k_voc_may.csv


In [4]:
bboxes_df = pd.read_csv(root_nas /'bboxes/dadet100k/dadet100kv5_bboxes.csv')
# hwpedia_del = pd.read_csv('./data/exps/dadetGallv4_del.csv')
# hwpedia_cat = pd.read_csv('./data/hwpedia_bbox.csv')
bboxes_df = bboxes_df[bboxes_df['label_pred']>=0]
# hwpedia_cat.to_csv('./data/hwpedia_bbox.csv', index=False)

In [5]:
bboxes_df.head()

,label_pred,bx1,bx2,bx3,bx4,bbox_ind,label_sp,stem,label_tp,mode,img_sp,source,img_tp
0,0,0.440030,0.904297,0.139430,0.060547,0,/home/chenyi/workspace/dataset/labels/labels6/...,551df69f15ca760f19dd6f40bfda2aa7,/home/chenyi/workspace/yolov5/datasets/dadet10...,train,/home/chenyi/workspace/dataset/data/data_fashi...,data_fashionpedia,/home/chenyi/workspace/yolov5/datasets/dadet10...
1,0,0.592954,0.879395,0.115442,0.073242,1,/home/chenyi/workspace/dataset/labels/labels6/...,551df69f15ca760f19dd6f40bfda2aa7,/home/chenyi/workspace/yolov5/datasets/dadet10...,train,/home/chenyi/workspace/dataset/data/data_fashi...,data_fashionpedia,/home/chenyi/workspace/yolov5/datasets/dadet10...
2,3,0.828336,0.753906,0.103448,0.101562,2,/home/chenyi/workspace/dataset/labels/labels6/...,551df69f15ca760f19dd6f40bfda2aa7,/home/chenyi/workspace/yolov5/datasets/dadet10...,train,/home/chenyi/workspace/dataset/data/data_fashi...,data_fashionpedia,/home/chenyi/workspace/yolov5/datasets/dadet10...
3,2,0.787856,0.651855,0.157421,0.127930,3,/home/chenyi/workspace/dataset/labels/labels6/...,551df69f15ca760f19dd6f40bfda2aa7,/home/chenyi/workspace/yolov5/datasets/dadet10...,train,/home/chenyi/workspace/dataset/data/data_fashi...,data_fashionpedia,/home/chenyi/workspace/yolov5/datasets/dadet10...
4,5,0.506840,0.487305,0.953488,0.849609,0,/home/chenyi/workspace/dataset/labels/labels6/...,bdca691551d3d1b5d95c6ad47b199152,/home/chenyi/workspace/yolov5/datasets/dadet10...,train,/home/chenyi/workspace/dataset/data/data_fashi...,data_fashionpedia,/home/chenyi/workspace/yolov5/datasets/dadet10...


In [9]:
from tqdm import tqdm
from PIL import Image
w_list = []
h_list = []
for p in tqdm(bboxes_df['img_sp'].values):
    img = Image.open(p)
    w,h = img.size
    w_list.append(w)
    h_list.append(h)
    # break

100%|█████████████████████████████████| 312242/312242 [00:26<00:00, 11822.65it/s]


In [10]:
bboxes_df['width'] = w_list
bboxes_df['height'] = h_list

In [11]:
# bboxes_df.to_csv(root_nas /'bboxes/dadet100k/dadet100kv5_bboxes.csv', index=False)

In [16]:
hwpedia_cat['stem'] = hwpedia_cat['img_sp'].apply(lambda x: Path(x).stem)

In [24]:
df_val = hwpedia_cat[hwpedia_cat['mode']=='val']
len(df_val)

32183

In [25]:
df_train = hwpedia_cat[hwpedia_cat['mode']=='train']
len(df_train)

187392

In [27]:
187392/(187392+50000)

0.7893779065848892

In [20]:
df_train_sample = df_train.sample(int(len(df_train)-5e4))

In [21]:
df = pd.concat([df_train_sample, df_val], axis=0)

In [22]:
# df.to_csv('./data/exps/dadetGallv4_del.csv', index=False)